In [ ]:
import random
def get_dummy_followings(handle):
    ret = set();
    while len(ret) < 6:
        ret.add(random.randint(1,9))
    return ret


In [ ]:
from tweety import Twitter, exceptions_

# read cridentials from file
# expecting 2 lines: email and passwd
with open("cridentials.txt", "r") as file:
        cridentials = [line.strip() for line in file.readlines()]

assert(len(cridentials) == 2)

tweety_app = Twitter("lesgo")
tweety_app.sign_in(cridentials[0], cridentials[1])

In [ ]:
#tweety_app.start()

In [ ]:
def get_tweety_followings(handle):
    try:
        # purpusefully fetch first page, beacuse anyways all new followings appear here
        fol = tweety_app.get_user_followings(handle);
        return set([f.username for f in fol.users])
    except exceptions_.UserNotFound as e:
        return set();

In [ ]:
# get a list of twitter handles, which will be checked for diff
# treats each line in a file as a separate handle
import random

def get_interesting_accs():
    with open("whitelist.txt", "r") as file:
        lines = [line.strip() for line in file.readlines()]
    random.shuffle(lines)
    return lines

In [ ]:
# the output function. for convenience
from datetime import datetime

def report(message, elements=None):
    if (elements is not None):
        print(message)
        for element in elements:
            print('https://twitter.com/' + element)         
        message += str(elements)
        
    with open('log.txt', 'a') as file:
        file.write(f'{datetime.now()}|{message}\n')

In [ ]:
import time
# prev_measures - dict [handle -> set of strings]
# get_following_function - f(handle) -> set of strings
def measure_diff(prev_measures, get_following_function):
    print('maesuring following set diff with ', get_following_function.__name__)
    # done again on every diff measurement, because the list can get updated
    # TODO remove no longer tracked ones from dict
    handles = get_interesting_accs();
    amount_of_changes = 0;
    
    for handle in handles:
        new_measurement = get_following_function(handle);
        new_elements = new_measurement - prev_measures.get(handle, set());

        if handle not in prev_measures:
            report(f'[{handle}] new handle recognised')
        elif len(new_elements) > 0:
            report(f'[{handle}] new subscribtions: ', new_elements)
            amount_of_changes += 1;
            
        prev_measures[handle] = new_measurement;
        time.sleep(random.randint(3,15))

    if amount_of_changes == 0:
        report('No changes since last time');

In [ ]:
# serialization/deserialization of following state between launches
import json
import shutil
import os

# Function to convert set into list
def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError

# Function to convert list back into set
def set_decoder(dict_obj):
    for key, value in dict_obj.items():
        if isinstance(value, list):
            dict_obj[key] = set(value)
    return dict_obj

# Serialization
def serialize_measures(collection, destination_filename, backup=True):
    if (backup and os.path.exists(destination_filename)):
        print(f'{destination_filename} already exists, backing up')
        shutil.move(destination_filename, destination_filename + '.backup')
    
    with open(destination_filename, 'w') as file:
        file.write(json.dumps(collection, default=set_default))


# Deserialization
def deserialize_collection(filename):
    result = dict()
    with open(filename, 'r') as file:
        result = json.loads(file.read(), object_hook=set_decoder)
    return result
#dict_obj_deserialized = json.loads(json_obj, object_hook=set_decoder)
#print("Deserialized Dictionary:", dict_obj_deserialized)


In [ ]:
#there'll be more functions to choose from (tweety, tweepy)
measures_file = 'serialized_measures.json';
prev_measures = deserialize_collection(measures_file)

assert(len(prev_measures) > 0)

measure_diff(prev_measures, get_tweety_followings)
serialize_measures(prev_measures, measures_file)
    
